# Code to plot the downloaded ncdf MSG file for testing

In [13]:
import numpy as np
import os
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cfeature           # import features
import uuid
from scipy import interpolate
import scipy.io
from glob import glob
import os
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Rectangle


def f_calculate_BT(MSG_data):
    
    ### constants taken from www-cdn.eumetsat.int/files/2020-04/pdf_effect_rad_to_brightness.pdf
    scale_factor = 0.2050356762076601
    add_offset = -10.456819486590666
    kB = 1.3806488*10**(-23)   ## Boltzmann constant [J/K]
    h = 6.62606957*10**(-34) ## Planck constant [J*s]
    c = 299792458         ## speed of light [m/s]
    ## lambda <- 10.8 * 10^(-6)  ## [µm] to [m]
    vc = 930.647 ## [1/cm]; for IR 10.8 channel Meteosat 8 (MSG FM-1); Table 7.2 doc^^
    vcm = vc * 100 ## [1/m] conversion of the wavenumber in SI units
    alpha = 0.9983  ## [] dimensionless, Table 7.2
    beta = 0.625   ## [K]; Table 7.2
    c1  = 2*h*c**2 ## [J*m^2/s] = [Wm2]
    c2 = h*c/kB ## [K*m]

    
    # converting the radiance in SI standard ( W/m-1/sr)
    rad_108_MKS = MSG_data.channel_9.values*10**(-5)  # radiance in W m-1 sr-1

    # converting radiance to brightness temperature [ in K]
    numerator = c2*vcm
    fraction = c1*(vcm**3)/rad_108_MKS
    denominator = alpha * (np.log(fraction))
    TB_108 = numerator/denominator - beta/alpha  ## [K]
    
    return(TB_108)

def plot_msg(lons, lats, variable, label, cmap='Greys', vmin=None, vmax=None, title=None, path_out=None):
    # Plot the map of MSG channels vs lat/lon
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.spines["top"].set_linewidth(3)
    ax.spines["right"].set_linewidth(3)
    ax.spines["bottom"].set_linewidth(3)
    ax.spines["left"].set_linewidth(3)
    ax.set_extent(extent_param)
    if title is not None:
        ax.set_title(title)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'fontsize': 14}
    gl.ylabel_style = {'fontsize': 14}

    pc = ax.pcolormesh(lons,lats,variable, cmap=cmap, vmin=vmin if vmin is not None else np.min(variable), 
                       vmax=vmax if vmax is not None else np.max(variable))
    #vmin, vmax=220.,
    plt.tick_params(axis='both',which='major',labelsize=14)

    ax.set_xlabel('Latitude [$^{\circ}$]')
    ax.set_ylabel('Longitude [$^{\circ}$]')
    ax.tick_params(which='minor', length=5, width=2)
    ax.tick_params(which='major', length=7, width=3)
    cbar = plt.colorbar(pc,ax=ax,shrink=0.75)
    cbar.set_label(label,fontsize=14)
    cbar.ax.tick_params(labelsize=14)

    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.5, color='orange')
    ax.add_feature(cfeature.STATES, linewidth=0.2)
    ax.add_feature(cfeature.BORDERS, linewidth=1., color='orange')

    if path_out is not None:
        plt.savefig(path_out, bbox_inches='tight', transparent=True)
        print('file saved')
    plt.show()
    plt.close()


In [11]:
MSG_file = 'HRSEVIRI_20230726T171200Z_20230726T171200Z_epct_3683354b_F.nc'

MSG_data = xr.open_dataset(MSG_file)
TB_108 = f_calculate_BT(MSG_data)

/var/folders/vq/pfv4n2796kvc782gmx0q4bhc0000gn/T/ipykernel_3390/513421538.py:40: RuntimeWarning: invalid value encountered in log
  denominator = alpha * (np.log(fraction))


In [12]:
TB_108 

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [32]:
# Read the longitude and latitute.
lons = MSG_data.x.values
lats = MSG_data.y.values

# define area
area_str = 'IDEA-S4S'; area_name = 'Alpine domain'
minlon = 5.; maxlon = 16.; minlat = 42.; maxlat = 51.5
# smaller domain which Paula used plotting DCs data
# minlon = 5.; maxlon = 16.; minlat = 44.5; maxlat = 49.5
extent_param = [minlon, maxlon, minlat, maxlat]

# extracting date from the string
datetime_str = os.path.basename(MSG_file).split('_')
print(datetime_str[1])
year = datetime_str[1][0:4]
month = datetime_str[1][4:6]
day = datetime_str[1][6:8]
hours = datetime_str[1][9:11]
minutes = datetime_str[1][11:13]
seconds = datetime_str[1][13:15]

print(f"{year}-{month}-{day}, {hours}:{minutes}:{seconds}")

20230726T171200Z
2023-07-26, 17:12:00


In [33]:
# figure_path_out = path_out+date[0:8]+'_'+num+'.png'
plot_msg(lons, lats, TB_108, 'Brightness temperature [K]', cmap='Greys', vmin=None, vmax=None,
         title=f"{year}-{month}-{day}, {hours}:{minutes}:{seconds}", path_out=None)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

<Figure size 1000x1000 with 0 Axes>

In [30]:
MSG_data

<xarray.Dataset>
Dimensions:        (x: 3712, y: 3712)
Coordinates:
  * x              (x) float64 -5.564e+06 -5.561e+06 ... 5.561e+06 5.564e+06
  * y              (y) float64 -5.564e+06 -5.561e+06 ... 5.561e+06 5.564e+06
Data variables:
    geostationary  |S1 ...
    channel_1      (y, x) float32 ...
    channel_2      (y, x) float32 ...
    channel_3      (y, x) float32 ...
    channel_4      (y, x) float32 ...
    channel_5      (y, x) float32 ...
    channel_6      (y, x) float32 ...
    channel_7      (y, x) float32 ...
    channel_8      (y, x) float32 ...
    channel_9      (y, x) float32 -10.46 -10.46 -10.46 ... -10.46 -10.46 -10.46
    channel_10     (y, x) float32 ...
    channel_11     (y, x) float32 ...
Attributes: (12/45)
    Conventions:                    CF-1.5
    ch01_cal:                       -1.114656032994e+00 2.185600064695e-02
    ch02_cal:                       -1.465775717050e+00 2.874070033431e-02
    ch03_cal:                       -1.211260244250e+00 2.375020086765e-02
    ch04_cal:                       -1.865920103496e-01 3.658666869601e-03
    ch05_cal:                       -4.242236706827e-01 8.318111189856e-03
    ...                             ...
    EPCT_roi:                       null
    EPCT_sensing_start:             null
    EPCT_sensing_stop:              null
    EPCT_software_ version:         3.2.0
    EPCT_software_name:             DataTailor
    EPCT_stretch_method:            null